In [ ]:
from flask import Flask, request, abort
from flask_restful import Resource, Api
from sqlalchemy import create_engine
from json import dumps
from flask.ext.jsonpify import jsonify
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import time
import os
import base64

import numpy as np
import tensorflow as tf
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import scripts.label_image
import argparse
import sys
import time
import os
import numpy as np
import tensorflow as tf
import json

from PIL import Image, ImageOps, ExifTags
import os, errno


def load_graph(model_file):
    graph = tf.Graph()
    graph_def = tf.GraphDef()

    with open(model_file, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def)

    return graph

def read_tensor_from_image_file(file_name, input_height=299, input_width=299, input_mean=0, input_std=255):
    input_name = "file_reader"
    output_name = "normalized"
    file_reader = tf.read_file(file_name, input_name)
    if file_name.endswith(".png"):
        image_reader = tf.image.decode_png(file_reader, channels = 3,
                                           name='png_reader')
    elif file_name.endswith(".gif"):
        image_reader = tf.squeeze(tf.image.decode_gif(file_reader,
                                                      name='gif_reader'))
    elif file_name.endswith(".bmp"):
        image_reader = tf.image.decode_bmp(file_reader, name='bmp_reader')
    else:
        image_reader = tf.image.decode_jpeg(file_reader, channels = 3,
                                            name='jpeg_reader')
    float_caster = tf.cast(image_reader, tf.float32)
    dims_expander = tf.expand_dims(float_caster, 0);
    resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
    normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
    sess = tf.Session()
    result = sess.run(normalized)
    return result

def load_labels(label_file):
    label = []
    proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label

def resize(file):
    desired_size = 256
 
    im = Image.open(file)
    
    try:
        for orientation in ExifTags.TAGS.keys():
            if ExifTags.TAGS[orientation]=='Orientation':
                break
        exif=dict(im._getexif().items())

        if exif[orientation] == 3:
            im=im.rotate(180, expand=True)
        elif exif[orientation] == 6:
            im=im.rotate(270, expand=True)
        elif exif[orientation] == 8:
            im=im.rotate(90, expand=True)

    except (AttributeError, KeyError, IndexError):
        # cases: image don't have getexif
        pass


    old_size = im.size  # old_size[0] is in (width, height) format
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    # use thumbnail() or resize() method to resize the input image
    # thumbnail is a in-place operation
    # im.thumbnail(new_size, Image.ANTIALIAS)
    im = im.resize(new_size, Image.ANTIALIAS)
    # create a new image and paste the resized on it
    new_im = Image.new("RGB", (desired_size, desired_size), "white")
    new_im.paste(im, ((desired_size-new_size[0])//2,
                        (desired_size-new_size[1])//2))
    new_im = new_im.convert('LA')
    # convert to RGB for easier saving
    new_im = new_im.convert('RGB')
    #new_im.rotate(180)

    new_im.save(file)



def createJSON(img):
    model_file = "tf_files/retrained_graph.pb"
    label_file = "tf_files/retrained_labels.txt"
    input_height = 224
    input_width = 224
    input_mean = 128
    input_std = 128
    input_layer = "input"
    output_layer = "final_result"
    jsons = []

    label_file = "tf_files/retrained_labels.txt"
    #for img in files:
    file_name = img
    # resize(file_name)
    graph = load_graph(model_file)
    t = read_tensor_from_image_file(file_name,
                                  input_height=input_height,
                                  input_width=input_width,
                                  input_mean=input_mean,
                                  input_std=input_std)

    input_name = "import/" + input_layer
    output_name = "import/" + output_layer
    input_operation = graph.get_operation_by_name(input_name);
    output_operation = graph.get_operation_by_name(output_name);

    with tf.Session(graph=graph) as sess:
        start = time.time()
        results = sess.run(output_operation.outputs[0],
                          {input_operation.outputs[0]: t})
        end=time.time()
    results = np.squeeze(results)

    top_k = results.argsort()[-5:][::-1]
    labels = load_labels(label_file)

    # print('\nEvaluation time (1-image): {:.3f}s\n'.format(end-start))
    print(img)
    labels = load_labels(label_file)
    print(labels)
    my_json_string = json.dumps({'car_back': str(results[0]), 
                                 'car_front': str(results[1]), 'car_left': str(results[2]),
                                 'car_right': str(results[3]),
                                 'car_wrong': str(results[4])})
    print(my_json_string)
    jsons = jsons + [my_json_string]
    
    res = [results[0], results[1], results[2], results[3], results[4]]
    # return(jsons[0].replace('\\"', '\"'))
    return(res)


app = Flask(__name__)
api = Api(app)


def evaluate(likelihoods, imageType):
    print(likelihoods)
    print(imageType)
    if (likelihoods[4] > 0.5):
        return(False)
    carSide = likelihoods[2] + likelihoods[3]
    carFrontBack = likelihoods[0] + likelihoods[1]
    
    if (imageType == "car_front") | (imageType == "car_back"):
        if (carFrontBack > 0.7):
            return(True)
    
    if (imageType == "car_side_left") | (imageType == "car_side_right"):
        if (carSide > 0.7):
            return(True)
    return(False)


class Classifier(Resource):
    def get(self):
        print("GET")
        print("##########")
        result = createJSON("temp.jpg")
        return(jsonify(result))
    def post(self):
        print("POST ANGLE")
        print("##########")
        if not request.json:
            abort(400)
        imageType = request.json['imageTypeId']
        image = request.json['image']
        #print(imageType)
            
        jpgtxt = base64.b64decode(image)

        f = open("temp.jpg", "bw")
        f.write(jpgtxt)
        f.close()
        result = createJSON("temp.jpg")
        isCorrect = evaluate(result, imageType)
        if (isCorrect == False):
            abort(400)
        else:
            return("Yaaay")
        #return(jsonify(result))
    
class LicencePlate(Resource):
    def post(self):
        print("POST LP")
        print("##########")
        if not request.json:
            abort(400)
        image = request.json['image']
        #print(imageType)
            
        jpgtxt = base64.b64decode(image)

        f = open("tempLP.jpg", "bw")
        f.write(jpgtxt)
        f.close()
        res = os.popen('alpr tempLP.jpg -c eu -p de').read()
        res = res.split("\n")
        res = res[1].split("\t")
        if res[0] == '':
            abort(409)
        else:
            res = res[0].split("- ")[1]
            print(res)
            return(res)

        
api.add_resource(Classifier, '/classifier')
api.add_resource(LicencePlate, '/licenceplate')


if __name__ == '__main__':
     app.run(host= '0.0.0.0', port='5002')


 * Running on http://0.0.0.0:5002/ (Press CTRL+C to quit)


POST ANGLE
##########


192.168.0.162 - - [13/Jan/2018 15:57:05] "POST /classifier HTTP/1.1" 200 -


temp.jpg
['std car back', 'std car front', 'std car left', 'std car right', 'std car wrong']
{"car_back": "3.57644e-05", "car_front": "0.997451", "car_left": "4.71533e-15", "car_right": "8.83384e-14", "car_wrong": "0.00251273"}
[3.5764366e-05, 0.99745148, 4.7153337e-15, 8.8338387e-14, 0.0025127262]
car_front
POST LP
##########


192.168.0.162 - - [13/Jan/2018 15:57:08] "POST /licenceplate HTTP/1.1" 200 -


...

temp.jpg
['std car back', 'std car front', 'std car left', 'std car right', 'std car wrong']
{"car_back": "2.57922e-11", "car_front": "1.20022e-10", "car_left": "0.00484767", "car_right": "0.995136", "car_wrong": "1.58459e-05"}
[2.5792193e-11, 1.2002156e-10, 0.0048476714, 0.99513644, 1.5845941e-05]
car_side_right
POST ANGLE
##########


192.168.0.162 - - [13/Jan/2018 16:23:03] "POST /classifier HTTP/1.1" 200 -


temp.jpg
['std car back', 'std car front', 'std car left', 'std car right', 'std car wrong']
{"car_back": "0.998699", "car_front": "0.000295398", "car_left": "1.08478e-17", "car_right": "3.98571e-12", "car_wrong": "0.00100518"}
[0.99869943, 0.00029539844, 1.084776e-17, 3.9857128e-12, 0.0010051822]
car_back
POST ANGLE
##########


192.168.0.162 - - [13/Jan/2018 16:23:23] "POST /classifier HTTP/1.1" 200 -


temp.jpg
['std car back', 'std car front', 'std car left', 'std car right', 'std car wrong']
{"car_back": "2.32499e-10", "car_front": "1.31783e-10", "car_left": "0.0010802", "car_right": "0.998918", "car_wrong": "2.0784e-06"}
[2.3249883e-10, 1.3178331e-10, 0.0010802015, 0.99891782, 2.0783959e-06]
car_side_left
